# SETUP

In [0]:
GDRIVE_ROOT = '/content/gdrive'
trainer = 'Vinh'

In [2]:
from google.colab import drive
drive.mount(GDRIVE_ROOT, force_remount=True)

Mounted at /content/gdrive


In [3]:
!rm -r triplet-cnn-cbir
!git clone https://github.com/VictorNM/triplet-cnn-cbir.git

Cloning into 'triplet-cnn-cbir'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 897 (delta 2), reused 4 (delta 2), pack-reused 889
Receiving objects: 100% (897/897), 59.16 MiB | 11.64 MiB/s, done.
Resolving deltas: 100% (527/527), done.


In [4]:
import sys
sys.path.append('triplet-cnn-cbir')

import os
import numpy as np

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.utils import Sequence

from src import data, models, visualization, utils, callbacks, experiment, database

Using TensorFlow backend.


In [0]:
DISSERTATION_ROOT = os.path.join(GDRIVE_ROOT, 'My Drive/bku-dissertation')
DATA_ROOT = os.path.join(DISSERTATION_ROOT, 'data')
DATASET_NAME = '_2_fan_kettle_600_200_200'

# RUN

## Load models

In [6]:
cnn_name = '2019-05-24 21:06:53'
cnn_path = os.path.join(DISSERTATION_ROOT, 'model', trainer, cnn_name + '.h5')
cnn_classifier = load_model(cnn_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [0]:
cnn_extractor = models.build_cnn_extractor(cnn_classifier)

In [8]:
# load triplet model
triplet_extractor_name = 'triplet-' + cnn_name
triplet_extractor_path = os.path.join(DISSERTATION_ROOT, 'model', trainer, triplet_extractor_name + '.h5')
triplet_extractor = load_model(triplet_extractor_path)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


## Load data

In [9]:
input_size = (224, 224)

test_path = '/content/gdrive/My Drive/bku-dissertation/data/raw/_2_fan_kettle_600_200_200/test'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_path, input_size)

Found 400 images belonging to 2 classes.


In [0]:
x_test, y_test = data.convert_generator_to_data(test_generator)

## Evaluate

In [11]:
scores = cnn_classifier.evaluate_generator(test_generator, steps=len(test_generator))
print('loss:', scores[0])
print('accuracy', scores[1])

loss: 0.19579541862010955
accuracy 0.94


In [13]:
print('Scores before triplet:')

mAP_normal = experiment.mAP_normal(cnn_extractor, x=x_test, y=y_test)
print('mAP without Kmeans:', mAP_normal)

mAP_kmeans = experiment.mAP_kmeans(cnn_extractor, x=x_test, y=y_test)
print('mAP with Kmeans:', mAP_kmeans)

ks = [10, 20, 30, 40, 50]
p_at_ks = []
for k in ks:
  p_at_k = experiment.mean_precision_at_k(
      extractor=cnn_extractor,
      x=x_test,
      y=y_test,
      k=k
  )
  p_at_ks.append(p_at_k)  
  print('P@{}: {}'.format(k, p_at_k))
  
for k in ks:
  p_at_k = experiment.mean_precision_at_k_kmeans(
      extractor=cnn_extractor,
      x=x_test,
      y=y_test,
      k=k
  )
  p_at_ks.append(p_at_k)  
  print('P@{}: {}'.format(k, p_at_k))

Scores before triplet:
P@10: 0.9230000000000002
P@20: 0.914375
P@30: 0.904
P@40: 0.8955625
P@50: 0.8905000000000001
P@10: 0.9222500000000001
P@20: 0.91375
P@30: 0.9015000000000001
P@40: 0.8927499999999999
P@50: 0.8870000000000001


In [0]:
triplet_extractor.summary()

In [16]:
print('Scores after triplet:')

mAP_normal = experiment.mAP_normal(triplet_extractor, x=x_test, y=y_test)
print('mAP without Kmeans:', mAP_normal)

mAP_kmeans = experiment.mAP_kmeans(triplet_extractor, x=x_test, y=y_test)
print('mAP with Kmeans:', mAP_kmeans)

ks = [10, 20, 30, 40, 50]
p_at_ks = []
for k in ks:
  p_at_k = experiment.mean_precision_at_k(
      extractor=triplet_extractor,
      x=x_test,
      y=y_test,
      k=k
  )
  p_at_ks.append(p_at_k)  
  print('P@{}: {}'.format(k, p_at_k))
  
for k in ks:
  p_at_k = experiment.mean_precision_at_k_kmeans(
      extractor=triplet_extractor,
      x=x_test,
      y=y_test,
      k=k
  )
  p_at_ks.append(p_at_k)  
  print('P@{}: {}'.format(k, p_at_k))

Scores after triplet:
P@10: 0.921
P@20: 0.9166249999999999
P@30: 0.91
P@40: 0.9051250000000001
P@50: 0.9007000000000001
P@10: 0.92
P@20: 0.9158750000000001
P@30: 0.9066666666666667
P@40: 0.8994374999999999
P@50: 0.8919
